In [22]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

exp = get_exp((38,7,42),directory_project)
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
hyph_anas_tip_hyph = [hyphat for hyphat in exp.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
chosen_RH = [rh for rh in RH if rh.ts[-1]==60]#+[rh for rh in BAS if len(rh.ts)>0 and rh.ts[-1]==60]
rh_not_anas = [rh for rh in chosen_RH if rh not in hyph_anas_tip_hyph]
rh_anas = [rh for rh in chosen_RH if rh in hyph_anas_tip_hyph]
stopping = []
for rh in rh_not_anas:
    if len(rh.ts)>2:
        t=rh.ts[-2]
        tp1= rh.ts[-1]
        pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
        lapse_time = get_time(exp, t, tp1)
        curv_growth = np.sum([get_length_um(seg) for seg in pixels])
        speed = curv_growth/lapse_time
        if speed<10:
            stopping.append(rh)
            
lapses_stopping= []
for rh in stopping:
    for index, t in enumerate(rh.ts[:-1]):
        tp1= rh.ts[index+1]
        pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
        lapse_time = get_time(exp, t, tp1)
        curv_growth = np.sum([get_length_um(seg) for seg in pixels])
        speed = curv_growth/lapse_time
        if speed<10:
            lapses_stopping.append(get_time(exp,rh.ts[0],t))
            break
            
lapse_anas = []
for hyph in hyph_anas_tip_hyph:
    if hyph in RH:
        for t in hyph.ts:
            if hyph.end.degree(t)>=3:
                time = get_time(exp,hyph.ts[0],t)
                lapse_anas.append((hyph.ts[0],time))
                break
                
lapses_anas = [lapse for t,lapse in lapse_anas]        
rh_not_stopping_or_anastomosing = [rh for rh in chosen_RH if rh not in stopping and rh not in rh_anas]
lapses_continuing = [get_time(exp,rh.ts[0],rh.ts[-1]) for rh in rh_not_stopping_or_anastomosing]
counts = []
lapses = [get_time(exp,0,t) for t in range (exp.ts)]
for lapse in lapses:
    counting=0
    for liste in [lapses_anas,lapses_stopping]:
        counting+=len([x for x in liste if x>=lapse])
    counts.append(counting)

begin = 2020-11-27 22:09:00 
  end = 2020-12-03 18:02:00
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in find

In [23]:
fig = plt.figure()
ax = fig.add_subplot(111)
init = 0
end = 60
def func2(t,alpha):
    return((np.exp(-alpha*(t))))
popt1, pcov = curve_fit(func2,lapses[init:end],np.array(counts)[init:end]/counts[init],bounds = ([0],1*[np.inf]),p0=[1])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(lapses[init:end],np.array(counts)[init:end]/counts[init], label = 'Observed proportion of hyphae not anastomosing')
ax.plot(lapses[init:end],func2(np.array(lapses),*popt1)[init:end],label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('lapse (h)',size=20)
ax.set_ylabel('P(t>lapse)',size =20)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
lifetimes = [get_time(exp, rh.ts[0], rh.ts[-1]) for rh in RH]
counts2 = [len([lifetime for lifetime in lifetimes if lifetime >=i]) for i in range(exp.ts)]
init = 0
end = 60
def func2(t,alpha):
    return((np.exp(-alpha*(t))))
popt1, pcov = curve_fit(func2,lapses[init:end],np.array(counts2)[init:end]/counts2[init],bounds = ([0],1*[np.inf]),p0=[1])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(lapses[init:end],np.array(counts2)[init:end]/counts2[init], label = 'Observed proportion of hyphae not anastomosing')
ax.plot(lapses[init:end],func2(np.array(lapses),*popt1)[init:end],label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('lapse (h)',size=20)
ax.set_ylabel('P(t>lapse)',size =20)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig = plt.figure()
ax = fig.add_subplot(111)
init = 0
end = 30
def func2(t,alpha):
    return((np.exp(-alpha*(t))))
popt1, pcov = curve_fit(func2,lapses[init:end],np.array(counts)[init:end]/np.array(counts2)[init:end]*(counts2[init]/counts[init]),bounds = ([0],1*[np.inf]),p0=[1])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(lapses[init:end],np.array(counts)[init:end]/np.array(counts2)[init:end]*(counts2[init]/counts[init]), label = 'Observed proportion of hyphae not anastomosing')
ax.plot(lapses[init:end],func2(np.array(lapses),*popt1)[init:end],label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('lapse (h)',size=20)
ax.set_ylabel('P(t>lapse)',size =20)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
[lifetime >=5 for lifetime in lifetimes]

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 Tru

In [2]:
bas_roots = [hyph.root.label for hyph in BAS]
rh_roots = [hyph.root.label for hyph in RH]
rh_length = 0
count_rh = 0
count_bas = 0
bas_length = 0
for rh in RH:
    length = rh.get_length_um(rh.ts[-1])
    nodes = rh.get_nodes_within(rh.ts[-1])[0]
    for node in nodes[1:-1]:
        if node in rh_roots:
            count_rh+=1
    rh_length += length
for rh in BAS:
    if len(rh.ts)>0:
        length = rh.get_length_um(rh.ts[-1])
        nodes = rh.get_nodes_within(rh.ts[-1])[0]
        for node in nodes[1:-1]:
            if node in rh_roots:
                count_bas+=1
        bas_length += length
times = [get_time(exp,0,t) for t in range(exp.ts)]

In [3]:
chosen_RH = [rh for rh in RH if rh.ts[-1]==60]#+[rh for rh in BAS if len(rh.ts)>0 and rh.ts[-1]==60]

In [9]:
plt.close('all')
t0s = {t:0 for t in range(exp.ts)}
for rh in RH:
    for t in rh.ts:
        t0s[t]+=1
    
# for rh in BAS:
#     for t in rh.ts:
#         t0s[t]+=1
nrh = list(t0s.values())
X = np.array(times[:]).reshape(-1,1)
y = np.log(np.array(nrh[:]))
reg = LinearRegression().fit(X, y)
# nrh= np.cumsum(list(t0s.values()))
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
# ax.scatter(times,np.array(nrh)/nrh[0])
# ax.plot(times,np.exp(times*(reg.coef_+0.002)))
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
end = 30

def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2,times[:end],np.array(nrh)[:end],bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(times[:end],np.array(nrh)[:end], label = 'Total number of apexes')
ax.plot(times[:end],func2(np.array(times),*popt1)[:end],label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('time (h)')
ax.set_ylabel('Number of RH')
plt.legend()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …